In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-data/test_clean_nonull_v2.csv
/kaggle/input/cleaned-data/train_clean_nonull_v2.csv
/kaggle/input/twitter-sentiment-analysis-batch-6/sample.csv
/kaggle/input/twitter-sentiment-analysis-batch-6/test1.csv
/kaggle/input/twitter-sentiment-analysis-batch-6/train1.csv


In [13]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score, classification_report
from scipy.sparse import hstack 

In [14]:
train=pd.read_csv('/kaggle/input/cleaned-data/train_clean_nonull_v2.csv')
test=pd.read_csv('/kaggle/input/cleaned-data/test_clean_nonull_v2.csv')
sample=pd.read_csv('/kaggle/input/twitter-sentiment-analysis-batch-6/sample.csv')
train.head()


,text,id,target,num_words,num_sentences,avg_lenght_words,count_Hashing,count_mention,count_number
0,thanks like always funny,1325379,1,14,2,4.900000,0,1,0
1,gone week pouting,275675,0,16,2,5.125000,0,0,0
2,stupid keep crashing cant update guess wait,583702,0,18,1,4.466667,0,0,0
3,ready weekend,1058019,1,6,1,4.200000,0,0,0
4,must great getting dont want always,1545430,1,19,1,4.533333,0,1,0


In [16]:
X_train_text = train['text']
X_train_numeric = train[['num_words', 'num_sentences', 'avg_lenght_words', 'count_Hashing', 'count_mention', 'count_number']]
y_train = train['target']

In [17]:
X_test_text = test['text']
X_test_numeric = test[['num_words', 'num_sentences', 'avg_lenght_words', 'count_Hashing', 'count_mention', 'count_number']]


In [18]:
tfidf = TfidfVectorizer(max_features=70000)
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

In [19]:
X_train_combined = hstack([X_train_tfidf, X_train_numeric])
X_test_combined = hstack([X_test_tfidf, X_test_numeric])

In [22]:
kmeans = KMeans(n_clusters=10, random_state=42)
train_clusters = kmeans.fit_predict(X_train_combined)
test_clusters = kmeans.predict(X_test_combined)


In [23]:
cluster_map = pd.DataFrame()
cluster_map['cluster'] = train_clusters
cluster_map['target'] = y_train

In [27]:
cluster_to_target = cluster_map.groupby('cluster')['target'].agg(lambda x:x.value_counts().index[0])

In [28]:
train_pred = cluster_map['cluster'].map(cluster_to_target)
train_pred = train_pred.astype(int)
# Calculate the F1 score on the training set
print("K-means Clustering Classification Report:")
print(classification_report(y_train, train_pred))
print("F1 Score on Training Data: ", f1_score(y_train, train_pred))
# Predict on the test set using the cluster mapping
test_pred = pd.Series(test_clusters).map(cluster_to_target)
test_pred = test_pred.astype(int)

K-means Clustering Classification Report:


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00    511860
           1       0.50      1.00      0.67    512139

    accuracy                           0.50   1023999
   macro avg       0.25      0.50      0.33   1023999
weighted avg       0.25      0.50      0.33   1023999

F1 Score on Training Data:  0.6667877495381274


In [29]:
submission = sample.copy()
submission['target'] = test_pred
# Save the submission file to the Kaggle working directory
submission.to_csv('/kaggle/working/submission.csv', index=False)
print("Submission file created successfully.")

Submission file created successfully.
